In [1]:
import pandas as pd
import matplotlib.pyplot as plt

#import findspark
#import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id, lit, when

from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, NaiveBayes, RandomForestClassifier
from pyspark.ml.feature import OneHotEncoder,StringIndexer,VectorAssembler
#from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.stat import ChiSquareTest
from pyspark.sql.types import *

from google.cloud import bigquery
from google.cloud import storage
#from google.colab import auth

import os

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./keys/key.json"

In [3]:
project_id = 'kltn-cloud-420208'
bigQuery_client = bigquery.Client(project=project_id)
storage_client = storage.Client()

In [4]:
query_factMatchStatistics = """
SELECT * FROM `kltn-cloud-420208.Football_DataWarehouse.Fact_Match_Statistics`
"""
query_dimMatch = """
SELECT * FROM `kltn-cloud-420208.Football_DataWarehouse.Dim_Match`
"""

query_dimTeam = """
SELECT * FROM `kltn-cloud-420208.Football_DataWarehouse.Dim_Team`
"""

In [5]:
match_dataset_model = pd.read_gbq(query_factMatchStatistics, project_id=project_id, dialect='standard')
dim_match = pd.read_gbq(query_dimMatch, project_id=project_id, dialect='standard')
dim_team = pd.read_gbq(query_dimTeam, project_id=project_id, dialect='standard')

In [6]:
match_dataset_model = match_dataset_model.rename(columns={'Match': 'Match_Key'})

In [7]:
match_dataset_model = match_dataset_model.drop(['Home_Total_Players_Stats', 'Home_Minutes', 
                                                'Away_Total_Players_Stats', 'Away_Minutes'], axis=1)

In [8]:
match_dataset_model_desc = pd.merge(match_dataset_model, dim_match[['Match_Key','Match_Date']], on='Match_Key', how='inner')



In [9]:
match_dataset_model_desc = match_dataset_model_desc.sort_values(by='Match_Date', ascending=False).reset_index(drop=True)

In [10]:

for col in match_dataset_model_desc.columns:
    if col != 'Match_Key' and col != 'Home_Team' and col != 'Away_Team' and match_dataset_model_desc[col].dtype == 'Int64':
        match_dataset_model_desc[col] = match_dataset_model_desc[col].astype(float)
        
match_dataset_model_desc['Home_Team'] = match_dataset_model_desc['Home_Team'].astype(str)
match_dataset_model_desc['Away_Team'] = match_dataset_model_desc['Away_Team'].astype(str)

In [11]:
list_result = ['Win' if home_score > away_score 
               else 'Draw' if home_score == away_score 
               else 'Lose' for home_score, away_score 
               in zip(match_dataset_model_desc['Home_Score'], match_dataset_model_desc['Away_Score'])]

In [12]:
match_dataset_model_desc['Result'] = list_result
match_dataset_model_desc

,Match_Key,Home_Team,Home_Possession,Home_Fouls,Home_Corners,Home_Crosses,Home_Aerials_Won,Home_Clearances,Home_Offsides,Home_Goal_Kicks,...,Home_Attack,Home_Midfield,Home_Defense,Away_Attack,Away_Midfield,Away_Defense,Home_Score,Away_Score,Match_Date,Result
0,12630,136,47.0,14.0,2.0,15.0,13.0,14.0,0.0,6.0,...,73.666667,74.333333,74.100000,74.444444,71.681818,71.272727,0.0,0.0,20240503.0,Draw
1,9988,93,43.0,2.0,2.0,14.0,4.0,4.0,0.0,4.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,20240503.0,Draw
2,12629,131,62.0,12.0,2.0,19.0,21.0,16.0,2.0,12.0,...,71.600000,72.625000,71.071429,69.000000,71.444444,70.625000,3.0,0.0,20240429.0,Win
3,7804,66,81.0,12.0,15.0,29.0,12.0,4.0,2.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.0,2.0,20240429.0,Win
4,9985,97,52.0,3.0,3.0,20.0,19.0,22.0,0.0,7.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,20240428.0,Draw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10534,384,12,65.0,8.0,7.0,27.0,13.0,15.0,1.0,8.0,...,70.800000,70.928571,71.142857,77.000000,75.375000,75.666667,1.0,3.0,20180811.0,Lose
10535,388,8,56.0,12.0,2.0,10.0,10.0,19.0,1.0,7.0,...,69.500000,71.133333,69.166667,75.166667,72.588235,74.166667,3.0,1.0,20180811.0,Win
10536,2601,36,58.0,8.0,3.0,25.0,21.0,24.0,2.0,1.0,...,0.000000,0.000000,0.000000,77.200000,78.454545,77.777778,2.0,2.0,20180811.0,Draw
10537,2595,30,47.0,11.0,2.0,12.0,9.0,36.0,4.0,5.0,...,85.600000,80.700000,79.454545,77.333333,78.083333,75.500000,2.0,1.0,20180810.0,Win


**Explain the ELO expression:**

The expression 1 / (1 + 10 ** ((rating2 - rating1) / 400)) is often used in the Elo rating system to calculate the expected outcome or probability of winning for a player in a two-player game or competition. Here's a breakdown of how this expression works:

- (rating2 - rating1) / 400: This part calculates the difference between the ratings of two players, rating2 and rating1. It divides the difference by 400. The division by 400 is a scaling factor used in the Elo rating system to adjust the impact of rating differences on the expected outcome.

- 10 ** ((rating2 - rating1) / 400): The result of step 1 is raised to the power of 10. This exponential calculation increases the impact of rating differences, as the larger the difference, the more the probability of winning or losing is affected.

- 1 + 10 ** ((rating2 - rating1) / 400): In this step, 1 is added to the result of step 2. This adjustment ensures that the denominator is always greater than 1, resulting in a value between 0 and 1 for the entire expression.

- 1 / (1 + 10 ** ((rating2 - rating1) / 400)): Finally, the expression is inverted by dividing 1 by the value obtained in step 3. The purpose of this inversion is to convert the value into a probability or expected outcome. The result will be a value between 0 and 1, representing the probability of the player with rating1 winning against the player with rating2 in the Elo rating system.

In summary, the expression 1 / (1 + 10 ** ((rating2 - rating1) / 400)) calculates the expected outcome or probability of winning for a player with rating1 against a player with rating2 in the Elo rating system, based on the difference between their ratings.

**What is K and 400:**

- K: The K constant represents the weight or magnitude of the rating update. It determines how much a player's rating will change based on the outcome of a match. A higher value of K will result in larger rating adjustments, while a lower value will lead to smaller adjustments. The value of 30 used in the code snippet is a common choice for K, but it can be adjusted according to the specific requirements of your Elo rating system.

- 400: The 400 constant is the standard scaling factor used in the Elo rating formula. It determines the sensitivity of the rating changes based on the difference in ratings between two players or teams. A larger value of 400 will result in smaller changes to ratings, while a smaller value will lead to more significant adjustments. The choice of 400 is a convention in the Elo rating system, but you can adjust it if desired.

In [18]:
import numpy as np
np.sort(match_dataset_model_desc["Home_Team"].unique().astype(int))

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  60,  61,  62,  63,  64,  65,  66,  68,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  95,  96,
        97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 108, 109, 110,
       111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123,
       124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136,
       137, 138, 139, 140, 141, 142, 143, 144, 145, 146])

In [25]:

class EloRatingSystem:
    def __init__(self,match_dataset_model,sens,K,draw_threshold):
        self.numOfTeam = match_dataset_model_desc['Home_Team'].nunique()
        self.ratings = pd.DataFrame({
            'Team_Key': [],
            'Elo': []
        })
        self.match_dataset_model = match_dataset_model.sort_values(by='Match_Date', ascending=True).reset_index(drop=True)
        self.sens = sens
        self.K = K
        self.draw_threshold = draw_threshold
    def update_ratings(self, team1, team2, result, sens, K):
        """
        Update Elo ratings for two teams after a match
        sens: Sensitivity parameter, higher value means ELO ratings will change more
        """
        elo1 = self.ratings.loc[self.ratings['Team_Key'] == team1, 'Elo'].values[0]
        elo2 = self.ratings.loc[self.ratings['Team_Key'] == team2, 'Elo'].values[0]
        # 10 power to amplify the Elo rating difference even further.
        # "1 + " to avoid division by zero
        # Expected gives the expected probability of winning for each team
        expected1 = 1 / (1 + 10 ** ((elo2 - elo1) / sens))
        expected2 = 1 / (1 + 10 ** ((elo1 - elo2) / sens))
        # Update Elo ratings
        if result == 0.5: 
            #If draw
            self.ratings.loc[self.ratings['Team_Key'] == team1, 'Elo'] = elo1 + K * (result - expected1)
            self.ratings.loc[self.ratings['Team_Key'] == team2, 'Elo'] = elo2 + K * (result - expected2)
        else:
            self.ratings.loc[self.ratings['Team_Key'] == team1, 'Elo'] = elo1 + K * (result - expected1)
            self.ratings.loc[self.ratings['Team_Key'] == team2, 'Elo'] = elo2 + K * (1 - result - expected2)
        
    def add_rating(self):
        for index, row in self.match_dataset_model.iterrows():
            if not self.ratings['Team_Key'].isin([row['Home_Team']]).any():
                self.ratings = pd.concat([self.ratings, pd.DataFrame({'Team_Key': [row['Home_Team']], 'Elo': [1500]})], ignore_index=True)
            if not self.ratings['Team_Key'].isin([row['Away_Team']]).any():
                self.ratings = pd.concat([self.ratings, pd.DataFrame({'Team_Key': [row['Away_Team']], 'Elo': [1500]})], ignore_index=True)

            home_elo = self.ratings.loc[self.ratings['Team_Key'] == row['Home_Team'], 'Elo'].values[0]
            away_elo = self.ratings.loc[self.ratings['Team_Key'] == row['Away_Team'], 'Elo'].values[0]
            self.match_dataset_model.loc[index, 'Home_Elo'] = home_elo
            self.match_dataset_model.loc[index, 'Away_Elo'] = away_elo
            if row['Result'] == 'Win':
                self.update_ratings(row['Home_Team'], row['Away_Team'], 1, self.sens, self.K)
            elif row['Result'] == 'Draw':
                self.update_ratings(row['Home_Team'], row['Away_Team'], 0.5, self.sens, self.K)
            else:
                self.update_ratings(row['Home_Team'], row['Away_Team'], 0, self.sens, self.K)
    

    def predict_outcome(self, team1, team2,elo1,elo2, sens, draw_threshold):
        expected1 = 1 / (1 + 10 ** ((elo2 - elo1) / self.sens))
        expected2 = 1 - expected1  # Since expected1 + expected2 = 1
        # Setting thresholds for prediction
        if abs(expected1 - expected2) < self.draw_threshold:  # Threshold can be adjusted
            return "Draw"
        elif expected1 > expected2:
            return "Win"
        else:
            return "Lose"
    def transform(self):
        self.add_rating()
        for index, row in self.match_dataset_model.iterrows():
            self.match_dataset_model.loc[index, 'Predicted_Result'] = self.predict_outcome(row['Home_Team'], row['Away_Team'],row['Home_Elo'],row['Away_Elo'], self.sens, self.draw_threshold)
        return self.match_dataset_model
    def measures(self, match_dataset_model):
        if 'Predicted_Result' not in match_dataset_model.columns:
            self.transform()
        match_dataset_model_measure = self.remove_first_match_of_each_team(match_dataset_model)
        numOfMatch = match_dataset_model_measure.shape[0]
        numOfCorrect = match_dataset_model_measure[match_dataset_model_measure['Result'] == match_dataset_model_measure['Predicted_Result']].shape[0]
        accuracy = numOfCorrect / numOfMatch
        return accuracy
    def remove_first_match_of_each_team(self,match_dataset_model):
        list_appear = []
        list_index_to_drop = []
        for index, row in match_dataset_model.iterrows():
            if row['Home_Team'] not in list_appear:
                list_index_to_drop.append(index)
                list_appear.append(row['Home_Team'])
                if row['Away_Team'] not in list_appear:
                    list_appear.append(row['Away_Team'])
            if len(list_appear) == self.numOfTeam:
                break
        match_dataset_model = match_dataset_model.drop(list_index_to_drop, axis=0).reset_index(drop=True)
        return match_dataset_model

    def reset_elo(self):
        self.ratings = pd.DataFrame({
            'Team_Key': [],
            'Elo': []
        })
    def reset_match_dataset(self):
        self.match_dataset_model = self.match_dataset_model.sort_values(by='Match_Date', ascending=True).reset_index(drop=True)
        self.match_dataset_model = self.match_dataset_model.drop(['Predicted_Result',"Home_Elo","Away_Elo"], axis=1)
    def optimize(self,sens_list, K_list, draw_threshold_list):
        best_accuracy = 0
        best_sens = 0
        best_draw_threshold = 0
        for sens in sens_list:
            for draw_threshold in draw_threshold_list:
                for K in K_list:
                    print("sens = ", sens, "K = ", K, "draw_threshold = ", draw_threshold)
                    self.reset_elo()
                    match_dataset_model = self.transform()
                    accuracy = self.measures(match_dataset_model)
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_sens = sens
                        best_draw_threshold = draw_threshold
                        best_K = K
                        with open("./measures/EloBestParameters.txt", "w") as file:
                            file.write(f"Best Accuracy: {best_accuracy}\n")
                            file.write(f"Best Sensitivity: {best_sens}\n")
                            file.write(f"Best Draw Threshold: {best_draw_threshold}\n")
                            file.write(f"Best K: {best_K}\n")
                        print("\n")
                        print("Best Accuracy: ", best_accuracy)
                        print("Best Sensitivity: ", best_sens)
                        print("Best Draw Threshold: ", best_draw_threshold)
                        print("Best K: ", best_K)
                        print("\n")
        return best_accuracy, best_sens, best_K, best_draw_threshold

In [21]:
elo = EloRatingSystem(match_dataset_model_desc, sens=50, K=50, draw_threshold=0.1)


In [22]:
elo_prediction = elo.transform()
elo_accuracy = elo.measures(elo_prediction)
print(elo_accuracy)

C:\Users\super\AppData\Local\Temp\ipykernel_20240\3307772068.py:57: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Draw' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.match_dataset_model.loc[index, 'Predicted_Result'] = self.predict_outcome(row['Home_Team'], row['Away_Team'],row['Home_Elo'],row['Away_Elo'], self.sens, self.draw_threshold)


0.44289133556725707


**Optimize**

In [ ]:
optimize_values = elo.optimize(
    sens_list=list(range(10, 100, 10)),
    K_list=list(range(10, 100, 10)),
    draw_threshold_list=np.arange(0.01, 0.50, 0.03)
)

sens =  10 K =  10 draw_threshold =  0.01


C:\Users\super\AppData\Local\Temp\ipykernel_22020\3387165464.py:57: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Draw' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.match_dataset_model.loc[index, 'Predicted_Result'] = self.predict_outcome(row['Home_Team'], row['Away_Team'],row['Home_Elo'],row['Away_Elo'], self.sens, self.draw_threshold)


sens =  10 K =  20 draw_threshold =  0.01
sens =  10 K =  30 draw_threshold =  0.01
sens =  10 K =  40 draw_threshold =  0.01
sens =  10 K =  50 draw_threshold =  0.01
sens =  10 K =  60 draw_threshold =  0.01
sens =  10 K =  70 draw_threshold =  0.01
sens =  10 K =  80 draw_threshold =  0.01
sens =  10 K =  90 draw_threshold =  0.01
sens =  10 K =  10 draw_threshold =  0.04
sens =  10 K =  20 draw_threshold =  0.04
sens =  10 K =  30 draw_threshold =  0.04
sens =  10 K =  40 draw_threshold =  0.04
sens =  10 K =  50 draw_threshold =  0.04
sens =  10 K =  60 draw_threshold =  0.04
sens =  10 K =  70 draw_threshold =  0.04
sens =  10 K =  80 draw_threshold =  0.04
sens =  10 K =  90 draw_threshold =  0.04
sens =  10 K =  10 draw_threshold =  0.06999999999999999
sens =  10 K =  20 draw_threshold =  0.06999999999999999
sens =  10 K =  30 draw_threshold =  0.06999999999999999
sens =  10 K =  40 draw_threshold =  0.06999999999999999
sens =  10 K =  50 draw_threshold =  0.06999999999999999
s

Current best value (0.4614289780814119, 50, 0.1)
(0.4614289780814119, 1, 0.01)
(0.4614289780814119, 50, 0.001)

In [24]:
optimize_values

NameError: name 'optimize_values' is not defined

# Test

In [58]:
elo = EloRatingSystem(match_dataset_model_desc, sens=0.01, K=100, draw_threshold=0.1)

In [59]:
elo_prediction = elo.transform()
elo_accuracy = elo.measures(elo_prediction)
print(elo_accuracy)

C:\Users\super\AppData\Local\Temp\ipykernel_22020\900771776.py:20: RuntimeWarning: overflow encountered in scalar power
  expected1 = 1 / (1 + 10 ** ((elo2 - elo1) / sens))
C:\Users\super\AppData\Local\Temp\ipykernel_22020\900771776.py:21: RuntimeWarning: overflow encountered in scalar power
  expected2 = 1 / (1 + 10 ** ((elo1 - elo2) / sens))
C:\Users\super\AppData\Local\Temp\ipykernel_22020\900771776.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Draw' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.match_dataset_model.loc[index, 'Predicted_Result'] = self.predict_outcome(row['Home_Team'], row['Away_Team'],row['Home_Elo'],row['Away_Elo'], self.sens, self.draw_threshold)


OverflowError: (34, 'Result too large')

In [ ]:
elo_prediction.to_csv('./data/elo_prediction.csv', index=False)